# Sistema de Compressão

$ \frac{d\dot{m}}{dt} = \frac{A_1}{L_C}(\phi (N(t), \dot{m})P_1 - P_P(t)) $

$ \frac{d P_P}{dt} = \frac{C_1^2}{\nu _P}(\dot{m}(t) - \alpha (t) K_\nu \sqrt{P_P - P_{out}}) $

$ \begin{matrix} A_1 & = & 2.6\centerdot 10^{-3} m² \\
\nu _P & = & 2.0 m³ \\
L_C & = & 2.0 m \\
K_\nu & = & \frac{0.38 kg}{(kPa)^{0.5}s} \\
P_1 & = & 4.5 MPa \\
P{out} & = & 5.0 MPa \end{matrix}
$

$ \frac{d\dot{m}}{dt} = \frac{2.6\centerdot 10^{-3}}{2.0}(1.5\centerdot 4.5 - P_P) $

$ \frac{d P_P}{dt} = \frac{479.029^2}{2.0}(\dot{m} - \alpha {0.38} \sqrt{P_P - 5.0}) $

#### Importações

In [57]:
import numpy as np
from scipy.optimize import fsolve
import casadi as ca
import plotly.graph_objects as go
import optuna
from plotly.subplots import make_subplots
import time
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from NumSol import Simulation
from Interpolacao import DataInterpolator


#### Constantes e Variáveis Auxiliares

In [49]:
np.random.seed(42)
print(np.random.seed)

# Constantes
A1 = (2.6)*(10**-3)
Lc = 2
kv = 0.38
P1 = 4.5
P_out = 5
C = 479

timestep = 3 # Passos no passado para prever o próximo
epochs = 5000
nData = 3000 
nAlphas = 5 # Número de vezes que o Alfa irá mudar, considere o treino e os testes.
alphas = np.random.uniform(0.35,0.65, nAlphas+1) # Abertura da válvula
perturb = 1e-4
tempo = 300
dt = 0.1 # Tempo amostral

# Variáveis auxiliares
interval = [np.linspace(i * tempo, (i + 1) * tempo, nData) for i in range(nAlphas)]
interval_test = [np.linspace(i * 60, (i + 1) * 60, 600) for i in range(nAlphas)]
massFlowrate = []
PlenumPressure = []
alpha_values = []
RNN_train = []
RNN_trainFut = []

<built-in function seed>


### Solução Numérica

##### Cálculo da Solução

In [58]:
# # Condições Iniciais
# def fun(variables, A1, Lc, kv, P1, P_out, C, alpha) :
#     (x,y) = variables
#     eqn_1 = (A1/Lc)* ((1.5 * P1) - y)*1e3
#     eqn_2 = (C**2)/2 * (x - alpha * kv * np.sqrt(y*1000 - P_out*1000))
#     return [eqn_1, eqn_2]

# result = fsolve(fun, (0, 10), args = (A1, Lc, kv, P1, P_out, C, alphas[0])) 

# init_m = result[0] 
# init_p = result[1]

# # Variáveis CasADi
# x = ca.MX.sym('x', 2)
# alpha = ca.MX.sym('alpha', 1)

# # Solução Numérica
# tm1 = time.time()
# for i in range(nAlphas):
#     alpha_values.append(alphas[i] + np.random.normal(0,perturb,nData))

#     rhs = ca.vertcat((A1/Lc)*((1.5 * P1) - x[1])*1e3, (C**2)/2 * (x[0] - alpha * kv * np.sqrt(x[1]*1000 - P_out*1000)))
#     ode = {'x' : x, 'ode' : rhs, 'p' : alpha }

#     F = ca.integrator('F','idas', ode, interval[0][0], dt)

#     for j in range(nData):
#         sol = F(x0 = [init_m, init_p], p = alpha_values[i][j])
#         xf_values = np.array(sol["xf"])
#         aux1, aux2 = xf_values
#         massFlowrate.append(aux1)
#         PlenumPressure.append(aux2)
#         init_m = aux1[-1]
#         init_p = aux2[-1]
#         RNN_train.append([aux1[0], aux2[0], alpha_values[i][j]])
#         RNN_trainFut.append([aux1[0], aux2[0], alpha_values[i][j]])
    
# tm2 = time.time()

# Crie uma instância da classe Simulation
sim = Simulation(A1, Lc, kv, P1, P_out, C, alphas, nAlphas, nData, perturb, tempo, dt)

# Execute a simulação
sim.run()

RNN_train = sim.RNN_train
RNN_trainFut = sim.RNN_trainFut
massFlowrate = sim.massFlowrate
PlenumPressure = sim.PlenumPressure

print(sim.time)
print(RNN_train)

TypeError: Simulation.__init__() takes 12 positional arguments but 13 were given

##### Gráfico do Modelo

In [5]:
fig = make_subplots(rows=1, cols=3, subplot_titles=("Vazão vs Tempo", "Pressão vs Tempo", "Alpha vs Tempo"))

for i in range(0, nAlphas):
    # Vazão
    fig.add_trace(go.Scatter(x=interval[i], y=np.squeeze(massFlowrate[i]), mode='lines',
                             name='Vazão', legendgroup='massflow', showlegend=i == 0), row = 1, col = 1)
    # Pressão
    fig.add_trace(go.Scatter(x=interval[i], y=np.squeeze(PlenumPressure[i]), mode='lines',
                             name='Pressão', legendgroup='pressure', showlegend=i == 0), row = 1, col = 2)
    # Alphas
    fig.add_trace(go.Scatter(x=interval[i], y=np.squeeze(alpha_values[i]), mode='lines', 
                             name='Alphas', line=dict(dash='dash'), legendgroup='alpha', showlegend=i == 0), row = 1, col = 3)

# Atualiza layout
fig.update_layout(
    xaxis_title='Tempo',
    grid=dict(rows=1, columns=3),
    template='plotly',
    showlegend=False
)

# Mostra a figura
fig.show()


### Rede Neural

##### Dados de Treino

In [33]:
RNN_train = np.array(RNN_train)


X_train = []
y_train = []

for i in range(len(RNN_train) - timestep):
    X_train.append(RNN_train[i:i + timestep])  
    if i + timestep < len(RNN_train):           
        y_train.append(RNN_train[i + timestep, :2])  

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)

x_min = X_train.amin(dim=(0, 1), keepdim=True)
x_max = X_train.amax(dim=(0, 1), keepdim=True)
print(x_min.shape)

y_train = y_train.unsqueeze(1)

torch.Size([1, 1, 3])


##### Rede

In [113]:
class MyModel(nn.Module):
    def __init__(self, units):
        super(MyModel, self).__init__()
        
        # Camada de entrada
        self.input_layer = nn.Linear(3, 3)
        
        # Camadas RNN
        self.rnn_layers = nn.ModuleList([
            nn.LSTM(input_size=3, hidden_size=units, batch_first=True, bidirectional=True, bias= True)
        ])
        self.rnn_layers.append(nn.LSTM(input_size=units*2, hidden_size=units, batch_first=True, bias= True))
        
        # Camada densa
        self.dense = nn.Linear(units, 2)

        # Inicialização dos pesos com Xavier (Glorot Uniform)
        self._initialize_weights()

    def _initialize_weights(self):
        # Inicializador Xavier (Glorot) nos pesos das camadas RNN
        for rnn_layer in self.rnn_layers:
            for name, param in rnn_layer.named_parameters():
                if 'weight' in name:
                    nn.init.xavier_uniform_(param)
                elif 'bias' in name:
                    nn.init.zeros_(param)

        nn.init.xavier_uniform_(self.dense.weight)
        nn.init.zeros_(self.dense.bias)

    def forward(self, inputs):
        # Passagem pelas camadas RNN
        rnn_output = 2 * (inputs - x_min) / (x_max - x_min) - 1
        for rnn_layer in self.rnn_layers:
            rnn_output, _ = rnn_layer(rnn_output)
            rnn_output = torch.tanh(rnn_output)  # Aplicando tanh explicitamente após cada camada RNN
         # Pegando apenas a última saída da sequência e desnormalizando
        dense_output = self.dense(rnn_output[:, -1, :])  # Dimensão [batch_size, hidden_size * num_directions]
        
        desnormalizado = ((dense_output + 1) / 2) * (x_max[:, :, :2] - x_min[:, :, :2]) + x_min[:, :, :2]
          # Pegando apenas a última saída da sequência
        return desnormalizado


def loss_custom(y_true, y_pred, inputs):
    # Implementação da função de perda
    m_t = (11* y_pred[:, :, 0] - 18 *inputs[:, -2, 0] + 9 * inputs[:, -3, 0] - 2 * inputs[:, 0, 0])/6*dt
    p_t = (11* y_pred[:, :, 1] - 18 *inputs[:, -2, 1] + 9 * inputs[:, -2, 1] - 2 * inputs[:, -2, 1])/6*dt
    fLoss_mass = torch.mean(torch.square(m_t - (A1/Lc)*((1.5 * P1) - y_pred[:, :, 1]) * 1e3))
    fLoss_pres = torch.mean(torch.square(p_t - (C**2)/2 * (y_pred[:, :, 0] - inputs[:, -1, -1]* kv * torch.sqrt((torch.abs(y_pred[:, :, 1] * 1000 - P_out * 1000))))))
    phys_loss = fLoss_mass + fLoss_pres
    data_loss =  torch.mean((y_true[:, 0, 0] - y_pred[:, :, 0]) ** 2) + torch.mean((y_true[:, 0, 1] - y_pred[:, :, 1]) ** 2)
    return data_loss +  1e-13*phys_loss

def train_model(model, train_loader, lr, epochs, optimizers):
    optimizer = optimizers(model.parameters(), lr=lr)
    model.train()
    
    for epoch in range(epochs):
        total_loss = 0
        for inputs, y_true in train_loader:
            optimizer.zero_grad()
            
            y_pred = model(inputs)
            loss = loss_custom(y_true, y_pred, inputs)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss / len(train_loader)}")




##### Treinamento

In [ ]:
def objective(trial):
    lr = trial.suggest_float('lr', 1e-6, 1e-3, log=True) 
    units = trial.suggest_int('units', 32, 64)
    batch_size = trial.suggest_int('batch_size', 16, 64) 
    num_layers = trial.suggest_int('num_layers', 0, 5)  

    model = create_model(lr, num_layers, units)

    model.fit(X_train, y_train, epochs=250, batch_size=batch_size, verbose=0)

    # Avaliação
    loss = model.evaluate(X_train, y_train, verbose=0)
    return loss

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=5)  # Ajuste o número de tentativas conforme necessário

# Exibir os melhores hiperparâmetros
print("Melhores hiperparâmetros: ", study.best_params)

In [121]:
model = MyModel(units = 50)

train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size= 32, shuffle=True)
Adam = optim.Adam
Rms = optim.RMSprop
train_model(model, train_loader, 1e-5, 300, Adam)

Epoch [1/300], Loss: 1.584164978598735
Epoch [2/300], Loss: 1.1881175854567017
Epoch [3/300], Loss: 0.6956012628352972
Epoch [4/300], Loss: 0.23599899221839174
Epoch [5/300], Loss: 0.03353626440678324
Epoch [6/300], Loss: 0.0069580834020358095
Epoch [7/300], Loss: 0.00522152495844516
Epoch [8/300], Loss: 0.0046217302679578695
Epoch [9/300], Loss: 0.004167770157192848
Epoch [10/300], Loss: 0.0037578254030135746
Epoch [11/300], Loss: 0.0033819125980130834
Epoch [12/300], Loss: 0.003058047236145925
Epoch [13/300], Loss: 0.002785932887142012
Epoch [14/300], Loss: 0.0025631716826867096
Epoch [15/300], Loss: 0.002374982638686681
Epoch [16/300], Loss: 0.00221185543003362
Epoch [17/300], Loss: 0.0020621471423921344
Epoch [18/300], Loss: 0.0019202302017196794
Epoch [19/300], Loss: 0.0017810179395378997
Epoch [20/300], Loss: 0.001646550235271862
Epoch [21/300], Loss: 0.0015226256308641145
Epoch [22/300], Loss: 0.001408479295653983
Epoch [23/300], Loss: 0.0013092178223950445
Epoch [24/300], Loss:

##### Dados de teste

In [142]:
massFlowrateTeste = []
PlenumPressureTeste = []
RNN_test = []
x_test = []
alpha_valuesTeste = []
alphasTeste = np.random.uniform(0.35,0.65, nAlphas) # Abertura da válvula
aux1 = []
aux2 = []

resultTeste = fsolve(fun, (0, 10), args = (A1, Lc, kv, P1, P_out, C, alphasTeste[0])) 

init_mT = resultTeste[0] 
init_pT = resultTeste[1]


tm1 = time.time()
for i in range(0,nAlphas):
    alpha_valuesTeste.append(alphasTeste[i] + np.random.normal(0,perturb,nData))

    rhs = ca.vertcat((A1/Lc)*((1.5 * P1) - x[1])*1e3, (C**2)/2 * (x[0] - alpha * kv * np.sqrt(x[1]*1000 - P_out*1000)))
    ode = {'x' : x, 'ode' : rhs, 'p' : alpha }
    
    for j in range(600):
        F = ca.integrator('F','idas', ode, interval[0][0], dt)
        sol = F(x0 = [init_mT, init_pT], p = alpha_valuesTeste[i][j])
        xf_values = np.array(sol["xf"])
        aux1, aux2 = xf_values
        massFlowrateTeste.append(aux1)
        PlenumPressureTeste.append(aux2)
        init_mT = aux1[-1]
        init_pT = aux2[-1]
        RNN_test.append([aux1[0], aux2[0], alpha_valuesTeste[i][j]])

tm2 = time.time()
print(tm2-tm1)

RNN_test = np.array(RNN_test)

for i in range(len(RNN_test) - 3):
    x_test.append(RNN_test[i:i + 3])

x_test = torch.tensor(x_test, dtype=torch.float32)



massFlowrateTeste = np.array(massFlowrateTeste)
PlenumPressureTeste = np.array(PlenumPressureTeste)
massFlowrateTeste = np.reshape(massFlowrateTeste, [nAlphas, 600])
PlenumPressureTeste = np.reshape(PlenumPressureTeste, [nAlphas, 600])

3.0114123821258545


##### Gráfico Rede Neural

In [143]:
fig = make_subplots(rows=1, cols=3, subplot_titles=("Loss","Mass Flow Rate vs Time", "Plenum Pressure vs Time"))


tm1 = time.time()
# Colocando o modelo em modo de avaliação
model.eval()

# Supondo que x_test já esteja definido e seja um tensor PyTorch
with torch.no_grad():  
    prediction = model(x_test)

mass = prediction[:, :, 0]
pressure = prediction[:, :, 1]

mass = mass.detach().numpy()
pressure = pressure.detach().numpy()

tm2 = time.time()
print(tm2-tm1)


interval3 = np.linspace(0, 300, 2997)

fig.add_trace(go.Scatter(x=interval3,y=np.squeeze(mass),mode='lines',
                                line=dict(dash='solid')),row=1, col=2)
fig.add_trace(go.Scatter(x=interval3,y=np.squeeze(pressure),mode='lines',
                                line=dict(dash='solid')),row=1, col=3)


for i in range(nAlphas):
    '''
    for j in range(0, len(interval2[i]), nData):
        end = j + nData
        if end > len(interval2[i]):
            end = len(interval2[i])
        
        fig.add_trace(go.Scatter(x=interval2[i][j:end],y=np.squeeze(mass[j:end]),mode='lines',
                                line=dict(dash='solid')),row=1, col=2)
        fig.add_trace(go.Scatter(x=interval2[i][j:end],y=np.squeeze(pressure[j:end]),mode='lines',
                                line=dict(dash='solid')),row=1, col=3)
    '''
    # Modelo
    fig.add_trace(go.Scatter(x=np.squeeze(interval_test[i]), y=np.squeeze(massFlowrateTeste[i]), mode='lines',name='Model Mass Flow Rate', line=dict(dash='dash', color='red')),
                  row=1, col=2)
    fig.add_trace(go.Scatter(x=np.squeeze(interval_test[i]), y=np.squeeze(PlenumPressureTeste[i]), mode='lines', name= 'Model Plenum Pressure', line=dict(dash='dash', color='red')),
                  row=1, col=3)

fig.update_layout(
    title='Resultados Rede Neural',
    xaxis_title='Time',
    yaxis_title='Value',
    template='plotly',
    showlegend=False
)
fig.show()


0.023540019989013672


In [144]:
model.eval()

massFlowrate100 = [x_test[0, 0, 0].item(), x_test[0, 1, 0].item(), x_test[0, 2, 0].item()]
PlenumPressure100 = [x_test[0, 0, 1].item(), x_test[0, 1, 1].item(), x_test[0, 2, 1].item()]

# Loop para fazer previsões
tm1 = time.time()
for i in range(2997):
    # Criação do tensor de entrada
    inputs = torch.tensor([[
        [massFlowrate100[-3], PlenumPressure100[-3], x_test[i, 0, 2].item()],
        [massFlowrate100[-2], PlenumPressure100[-2], x_test[i, 1, 2].item()],
        [massFlowrate100[-1], PlenumPressure100[-1], x_test[i, 2, 2].item()]
    ]], dtype=torch.float32)
    # Desabilitar o cálculo de gradientes
    with torch.no_grad():
        prediction100 = model(inputs)  # Chamar o modelo diretamente para previsões

    # Armazenar as previsões
    massFlowrate100.append(prediction100[0, 0, 0].item())  # Converte tensor para valor escalar  
    PlenumPressure100.append(prediction100[0, 0, 1].item())
tm2 = time.time()
print(tm2-tm1)


1.5207805633544922


In [145]:
fig3 = make_subplots(rows=1, cols=3, subplot_titles=("Loss","Mass Flow Rate vs Time", "Plenum Pressure vs Time"))

fig3.add_trace(go.Scatter(x=interval3,y=np.squeeze(massFlowrate100),mode='lines',
                                line=dict(dash='solid')),row=1, col=2)
fig3.add_trace(go.Scatter(x=interval3,y=np.squeeze(PlenumPressure100),mode='lines',
                                line=dict(dash='solid')),row=1, col=3)



for i in range(nAlphas):
    # Modelo
    fig3.add_trace(go.Scatter(x=np.squeeze(interval_test[i]), y=np.squeeze(massFlowrateTeste[i]), mode='lines',name='Model Mass Flow Rate', line=dict(dash='dash', color='red')),
                  row=1, col=2)
    fig3.add_trace(go.Scatter(x=np.squeeze(interval_test[i]), y=np.squeeze(PlenumPressureTeste[i]), mode='lines', name= 'Model Plenum Pressure', line=dict(dash='dash', color='red')),
                  row=1, col=3)

fig3.update_layout(
    title='Resultados Rede Neural',
    xaxis_title='Time',
    yaxis_title='Value',
    template='plotly',
    showlegend=False
)
fig3.show()